In [1]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%matplotlib inline
import warnings;warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import os

In [2]:
#### load data files ###

# Change file name prefix here and you are good to go:D
prefix = "train_"
# prefix = "A_"

# Generate data file name list
file_names = ["Application.csv","Personas.csv","History_Application.csv","History_Payment.csv"]
for idx, val in enumerate(file_names):
    file_names[idx] = prefix + val  

# load data files
d1 = pd.read_csv('./data/' + file_names[0])
d2 = pd.read_csv('./data/' + file_names[1])
d3 = pd.read_csv('./data/' + file_names[2])
d4 = pd.read_csv('./data/' + file_names[3])

# d1.head()
# d1.shape

# d2.head()
# d2.shape

# d3.head()
# d3.shape

# d4.head()
# d4.shape

In [3]:
### Merge d1 d2 ###
uid = '申请编号'
d12 = pd.merge(d1, d2, how='inner', on=uid)
d12.shape

(140000, 62)

In [4]:
### d12 Outlier：Get ###
low = .01
high = .99
quant_df = d12.drop([uid], axis=1).quantile([low, high]).T
quant_df.to_csv('./tmp/1_d12_outlier.csv')

In [5]:
### d12 数据插补 ###

# 距申请日期>0, 置空值
d12_cols = {}
d12_cols['_dt'] = [
    '最近一次换手机号码距申请日天数',
    '出生日期距申请日期天数',
    '工作日期距申请日期天数',
    '注册日期距申请日期天数',
    '身份认证日期距申请日期天数']

for col in d12_cols['_dt']:
    d12.loc[d12[col] > 0, col] = np.NaN

### d1 数据插补 ###

# 数值型缺失插补均值
d12.fillna({
    '外部评分':d12['外部评分'].mean()}
     , inplace=True)

# pd.Series(d12['贷款申请前1小时内征信查询次数']).mode()
# pd.Series(d12['贷款申请前1天内征信查询次数']).mode()
# pd.Series(d12['贷款申请前1周内征信查询次数']).mode()
# pd.Series(d12['贷款申请前1个月内征信查询次数']).mode()
# pd.Series(d12['贷款申请前1个季度内征信查询次数']).mode()
# pd.Series(d12['贷款申请前1年内征信查询次数']).mode()

# 计数型缺失插补众数
d12.fillna({
    '贷款申请前1小时内征信查询次数':0,
    '贷款申请前1天内征信查询次数':0,
    '贷款申请前1周内征信查询次数':0,
    '贷款申请前1个月内征信查询次数':0,
    '贷款申请前1个季度内征信查询次数':0,
    '贷款申请前1年内征信查询次数':0
    }, inplace=True)

# for col in d12.columns:
#     d1[col].value_counts(dropna=False)

### d2 数据插补 ###

# 计数型缺失插补众数
pd.Series(d12['孩子个数']).mode()
d12['孩子个数'].value_counts()
d12.loc[
    (d12['孩子个数'] == -1), '孩子个数'] = 0

0   -1
dtype: int64

-1     97711
 0     28100
 1     12255
 2      1672
 3       200
 4        43
 5        11
 6         5
 18        1
 10        1
 8         1
Name: 孩子个数, dtype: int64

In [6]:
### Outlier：Apply

outliers = pd.read_csv('./tmp/0_outlier1.csv', index_col=0)
outliers.shape
outliers.head()

def set_outlier(col):
    if col < col_min:
        col = col_min
    elif col > col_max:
        col = col_max
    return col

# def set_outlier(col):
#     if col < col_min or col > col_max:
#         col = np.NaN
#     return col
    
for col in outliers.index:
# for col in '出生日期距申请日期天数', '社交圈违约信息1_2':
    col_min = outliers.loc[col, 'min']
    col_max = outliers.loc[col, 'max']
    d12[col] = d12[col].apply(set_outlier)
    
# d12.to_csv(f'./tmp/2_eda_outlier.csv')

(61, 2)

,min,max
地址是否一致标志1,0,1
地址是否一致标志2,0,1
地址是否一致标志3,0,1
地址是否一致标志4,0,1
地址是否一致标志5,0,1


In [7]:
### d12 类型变量转dummies ###
d12_dum = d12

### d1
pd.Series(d12['陪同申请人']).mode()
d12_dum = pd.get_dummies(d12_dum, dummy_na=True, columns=[
    '陪同申请人'
])
d12_dum.loc[
    (d12_dum['陪同申请人_nan'] == 1), '陪同申请人_7.0'] = 1

d12_dum = pd.get_dummies(d12_dum, columns=[
    '申请周内日',
    '申请时点'
])

### d2

# 类型特征值太多的，低频值归other类
d12.loc[(d2['单位类型'] == 25) &
           (d12['单位类型'] == 49) &
           (d12['单位类型'] == 49) &
           (d12['单位类型'] == 18) &
           (d12['单位类型'] == 37) &
           (d12['单位类型'] == 23) &
           (d12['单位类型'] == 15) &
           (d12['单位类型'] == 52) &
           (d12['单位类型'] == 6) &
           (d12['单位类型'] == 32) &
           (d12['单位类型'] == 45) &
           (d12['单位类型'] == 29) &
           (d12['单位类型'] == 8) &
           (d12['单位类型'] == 17) &
           (d12['单位类型'] == 0) &
           (d12['单位类型'] == 36) &
           (d12['单位类型'] == 19) &
           (d12['单位类型'] == 44) &
           (d12['单位类型'] == 27) &
           (d12['单位类型'] == 10) &
           (d12['单位类型'] == 50) &
           (d12['单位类型'] == 22) &
           (d12['单位类型'] == 21) &
           (d12['单位类型'] == 9) &
           (d12['单位类型'] == 12) &
           (d12['单位类型'] == 14), '单位类型'] = 'other'

# d12['单位类型'].value_counts()

d12_dum = pd.get_dummies(d12_dum, columns=[
    '收入类型',
    '教育程度',
    '居住状态',
    '职业'
#     '单位类型'
])

# for col in d12_dum.columns:
#     print(col)

d12_dum.shape

0    7.0
dtype: float64

(140000, 119)

In [8]:
### d3数据清洗 ###

### 特征分类 ###
uid = '申请编号'
hid = '历史申请编号'
d3_cols = {}
d3_cols['no_fea'] = [uid, hid]

# 金额型
d3_cols['curr'] = [
    '贷款年金',
    '申请额度',
    '信用额度',
    '首付金额',
    '商品价格']

# 类别型
d3_cols['cat'] = [
    '贷款类型',
    '贷款用途',
    '合同状态',
    '付款方式',
    '拒绝原因',
    '陪同申请人',
    '客户类型',
    '商品类别',
    '组合类型',
    '产品类型',
    '获客渠道',
    '销售区域',
    '行业',
    '收益级别',
    '产品组合']

# 时点型
d3_cols['time_point'] = [
    '申请周内日',
    '申请时点']

# 时长型
d3_cols['time_span'] = [
    '历史贷款授信距本次申请时间',
    '首付时间']

# 布尔型
d3_cols['bool'] = [
    '是否为最后一次申请记录',
    '是否为当天最后一次申请记录']

# 概率型
d3_cols['prob'] = [
    '首付率',
    '利率1',
    '利率2']

# 计数型
d3_cols['cnt'] = [
    '贷款期数']

In [9]:
### d3数据清洗 - 金额型 ###
# d3_cols['curr'] = [
#     '贷款年金',
#     '申请额度',
#     '信用额度',
#     '首付金额',
#     '商品价格']

# ''' 查看含有缺失值字段 '''
# for col in d3_cols['curr']:
#     d3[col].value_counts(dropna=False)

# ''' 含有缺失值字段 '''
# nan_cols = [
#     '贷款年金',
#     '首付金额',
#     '商品价格'
# ]

# 数值型缺失插补均值
d3.fillna({
    '贷款年金':d3['贷款年金'].mean(),
    '首付金额':d3['首付金额'].mean(),
    '商品价格':d3['商品价格'].mean()}
     , inplace=True)

In [10]:
### d3数据清洗 - 时长型 ###
# d3_cols['time_span'] = [
#     '历史贷款授信距本次申请时间',
#     '首付时间']

# ''' 查看含有缺失值字段 '''
# for col in d3_cols['time_span']:
#     d3[col].value_counts(dropna=False)

# 数值型异常值置None
d3.loc[
    (d3['首付时间'] == 292204.0), '首付时间'] = None

# 数值型缺失均值
d3.fillna({
    '首付时间':d3['首付时间'].mean()}
     , inplace=True)

In [11]:
### d3数据清洗 - 概率型 ###
# d3_cols['prob'] = [
#     '首付率',
#     '利率1',
#     '利率2']

# ''' 查看含有缺失值字段 '''
# for col in d3_cols['prob']:
#     d3[col].value_counts(dropna=False)
    
# 数值型缺失均值
d3.fillna({
    '首付率':d3['首付率'].mean(),
    '利率1':d3['利率1'].mean(),
    '利率2':d3['利率2'].mean()}
     , inplace=True)

In [12]:
### d3数据清洗 - 类别型 ###
d3_dum = d3

# d3_cols['cat'] = [
#     '贷款类型',
#     '贷款用途',
#     '合同状态',
#     '付款方式',
#     '拒绝原因',
#     '陪同申请人',
#     '客户类型',
#     '商品类别',
#     '组合类型',
#     '产品类型',
#     '获客渠道',
#     '销售区域',
#     '行业',
#     '收益级别',
#     '产品组合']

# ''' 查看含有缺失值字段 '''
# for col in d3_cols['cat']:
#     d3[col].value_counts(dropna=False)

# ''' d3 - 类别型desc '''
# d3_cat_desc = d3[d3_cols['cat']].astype("object").describe(include=['object'])
# print(d3_cat_desc)

# ''' 含有缺失值字段 '''
# nan_cols = [
#     '贷款类型',
#     '陪同申请人',
#     '产品组合'
# ]

### 处理含空值的离散特征，转换dummies

# d3['贷款类型'].value_counts()
# d3['陪同申请人'].value_counts()
# d3['产品组合'].value_counts()

d3_dum = pd.get_dummies(d3_dum, dummy_na=True, columns=[
    '贷款类型',
    '陪同申请人',
    '产品组合'
])

# 空值取众数
d3_dum.loc[d3_dum['贷款类型_nan']==1, '贷款类型_0'] = 1
d3_dum.loc[d3_dum['陪同申请人_nan']==1, '陪同申请人_7.0'] = 1
d3_dum.loc[d3_dum['产品组合_nan']==1, '产品组合_2.0'] = 1

### 处理其他离散特征，有长尾分布, 小值归other类，转换dummies

# d3_dum['贷款用途'].value_counts()

d3_dum.loc[(d3_dum['贷款用途'] != 23) & (d3_dum['贷款用途'] != 24), '贷款用途'] = 'other'

# d3_dum['贷款用途'].value_counts()
# d3_dum['合同状态'].value_counts()
# d3_dum['付款方式'].value_counts()
# d3_dum['拒绝原因'].value_counts()
# d3_dum['客户类型'].value_counts()
# d3_dum['商品类别'].value_counts()

d3_dum.loc[
    (d3_dum['商品类别'] != 27) & 
    (d3_dum['商品类别'] != 19) &
    (d3_dum['商品类别'] != 7) &
    (d3_dum['商品类别'] != 5) &
    (d3_dum['商品类别'] != 2) &
    (d3_dum['商品类别'] != 11), '商品类别'] = 'other'

# d3_dum['组合类型'].value_counts()
# d3_dum['产品类型'].value_counts()
# d3_dum['获客渠道'].value_counts()
# d3_dum['销售区域'].value_counts()

d3_dum['销售区域_count'] = d3_dum.groupby('销售区域')['销售区域'].transform('count')
d3_dum.loc[d3_dum['销售区域_count'] < 5000, '销售区域'] = 'other'
# d3_dum = d3_dum.drop('销售区域_count', axis=1)

# d3_dum['行业'].value_counts()

d3_dum = pd.get_dummies(d3_dum, columns=[
    '贷款用途',
    '合同状态',
    '付款方式',
    '拒绝原因',
    '客户类型',
    '商品类别',
    '组合类型',
    '产品类型',
    '获客渠道',
    '销售区域',
    '行业'
])

d3_dum.shape

(647963, 121)

In [13]:
### 数据清洗 - 时点型 ###

# 时点型(类别)
# d3_cols['time_point'] = [
#     '申请周内日',
#     '申请时点']

# ''' 查看含有缺失值字段 '''
# for col in d3_cols['time_point']:
#     d3_dum[col].value_counts(dropna=False)

### 处理含空值的离散特征，转换dummies

# d3_dum['申请周内日'].value_counts()
# d3_dum['申请时点'].value_counts()

d3_dum.loc[
    (d3_dum['申请时点'] != 3) & 
    (d3_dum['申请时点'] != 4) &
    (d3_dum['申请时点'] != 5) &
    (d3_dum['申请时点'] != 6) &
    (d3_dum['申请时点'] != 7) &
    (d3_dum['申请时点'] != 8) &
    (d3_dum['申请时点'] != 9), '申请时点'] = 'other'

d3_dum = pd.get_dummies(d3_dum,columns=[
    '申请周内日',
    '申请时点'
])

In [15]:
### d3 group by ###

agg_dict = {}

# 数值型 - 金额
curr_agg = ['mean', 'max', 'min']
for col in d3_cols['curr']:
    agg_dict[col] = curr_agg

# 数值型 - 时长 
time_span_agg = ['mean', 'max', 'min']
for col in d3_cols['time_span']:
    agg_dict[col] = time_span_agg

# 数值型 - 概率
prob_agg = ['mean', 'max', 'min']
for col in d3_cols['prob']:
    agg_dict[col] = prob_agg

# 离散型 - 类型
cat_agg = ['sum', 'mean']
for col1 in d3_dum.columns:
    for col2 in d3_cols['cat']:
        if (col2 in col1):
            agg_dict[col1] = cat_agg

# 离散型 - 时点
time_point_agg = ['sum', 'mean']
for col1 in d3_dum.columns:
    for col2 in d3_cols['time_point']:
        if (col2 in col1):
            agg_dict[col1] = time_point_agg


# print(agg_dict)

d3_dum = d3_dum.groupby('申请编号').agg(agg_dict)

d3_dum.columns = pd.Series(d3_dum.columns.tolist()).apply(pd.Series).sum(axis=1)

d3_dum.shape

In [19]:
### Merge d12_dum and d3_dum ###
d12_d3_dum = pd.merge(d12_dum, d3_dum, how='left', on='申请编号')
d12_d3_dum.shape
d12_d3_dum.head()



d12_d3_dum.to_csv(f'./tmp/' + prefix + '_d12_d3_dum.csv')


(140000, 387)

,申请编号,贷款类型,信用额度,贷款年金,商品价格,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,身份认证日期距申请日期天数,是否提供手机号,...,申请时点_6sum,申请时点_6mean,申请时点_7sum,申请时点_7mean,申请时点_8sum,申请时点_8mean,申请时点_9sum,申请时点_9mean,申请时点_othersum,申请时点_othermean
0,0,0,460190.889355,17463.042019,419951.511045,-10107.0,-342.0,-5421.0,-3292.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,1,0,424370.659603,15585.046388,384597.975692,-13980.0,-1110.0,-3387.0,-826.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
2,2,0,469330.587153,18837.558252,419951.511045,-13331.0,-2246.0,-3870.0,-171.0,1,...,1.0,0.1,7.0,0.7,1.0,0.1,1.0,0.1,0.0,0.0
3,3,0,464188.353140,16754.587069,409850.500944,-16540.0,NaN,-970.0,-2916.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,434196.988738,16165.009200,394698.985793,-17919.0,-11037.0,-9350.0,-3588.0,1,...,1.0,0.2,0.0,0.0,2.0,0.4,1.0,0.2,0.0,0.0
